# Temporal Analysis: Artist Evolution

## Introduction

Artists don't stand still. Their palettes **evolve** over decades—sometimes gradually, sometimes dramatically. Picasso's somber Blue Period gave way to the warmer Rose Period. Monet's late Water Lilies became increasingly abstract and colorful.

In this final lesson, we add the **temporal dimension** to our color analysis, tracking how artists and movements changed over time.

### What You'll Learn

1. **Career Trajectory Analysis**: Track palette evolution across an artist's life
2. **Period Detection**: Automatically identify artistic phases (Blue Period, etc.)
3. **Movement Timeline**: How color trends evolved across art history
4. **Predictive Modeling**: Predict artwork date/period from color features
5. **Change Point Detection**: Find moments of dramatic stylistic shift

### Famous Artistic Evolutions

| Artist | Evolution | Color Signature |
|--------|-----------|----------------|
| **Picasso** | Blue → Rose → Cubism → Classical | Dramatic temperature shifts |
| **Monet** | Realism → Impressionism → Late abstraction | Increasing brightness |
| **Van Gogh** | Dark Dutch → Bright French | Saturation explosion |
| **Rothko** | Figurative → Color Field | Simplification, intensity |

Let's trace the color journeys of the masters!

## Setup

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Renoir imports
from renoir import ArtistAnalyzer
from renoir.color import ColorExtractor, ColorAnalyzer, ColorVisualizer

# ML imports
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, classification_report
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
import seaborn as sns

# Initialize renoir components
artist_analyzer = ArtistAnalyzer()
color_extractor = ColorExtractor()
color_analyzer = ColorAnalyzer()
visualizer = ColorVisualizer()

# Load dataset
print("Loading WikiArt dataset...")
dataset = artist_analyzer._load_dataset()
print(f"Loaded {len(dataset)} artworks")

# Get metadata
artist_names = dataset.features['artist'].names
style_names = dataset.features['style'].names
print(f"Artists: {len(artist_names)}, Styles: {len(style_names)}")

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100

# Reproducibility
SEED = 42
np.random.seed(SEED)

## Part 1: Extracting Temporal Data

First, we need to extract artworks with date information and compute color features.

In [ ]:
def extract_color_features(image, n_colors=8):
    """
    Extract comprehensive color features from an artwork.
    """
    try:
        palette = color_extractor.extract_dominant_colors(image, n_colors=n_colors)
        if not palette or len(palette) < 3:
            return None, None
        
        stats = color_analyzer.analyze_palette_statistics(palette)
        temp = color_analyzer.analyze_color_temperature_distribution(palette)
        harmony = color_analyzer.analyze_color_harmony(palette)
        
        hsv_colors = [color_analyzer.rgb_to_hsv(c) for c in palette]
        hues = [h[0] for h in hsv_colors]
        sats = [h[1] for h in hsv_colors]
        vals = [h[2] for h in hsv_colors]
        
        reds = [c[0] for c in palette]
        greens = [c[1] for c in palette]
        blues = [c[2] for c in palette]
        
        features = {
            'mean_hue': stats['mean_hue'],
            'mean_saturation': stats['mean_saturation'],
            'mean_brightness': stats['mean_value'],
            'mean_red': np.mean(reds),
            'mean_green': np.mean(greens),
            'mean_blue': np.mean(blues),
            'std_hue': np.std(hues),
            'std_saturation': np.std(sats),
            'std_brightness': np.std(vals),
            'warm_ratio': temp['warm_percentage'] / 100,
            'cool_ratio': temp['cool_percentage'] / 100,
            'color_diversity': color_analyzer.calculate_color_diversity(palette),
            'harmony_score': harmony.get('harmony_score', 0),
            'brightness_range': max(vals) - min(vals),
            'saturation_range': max(sats) - min(sats),
        }
        
        return features, palette
    except:
        return None, None


def collect_artist_timeline(dataset, artist_name, min_works=20):
    """
    Collect artworks with dates for a specific artist.
    
    Returns DataFrame with features and dates.
    """
    # Find artist index
    artist_key = artist_name.lower().replace(' ', '-')
    artist_idx = None
    
    for idx, name in enumerate(artist_names):
        if artist_key in name.lower():
            artist_idx = idx
            break
    
    if artist_idx is None:
        print(f"Artist '{artist_name}' not found")
        return None
    
    # Collect works
    works_data = []
    
    for item in dataset:
        if item['artist'] == artist_idx:
            # Try to extract year from date field
            try:
                year = int(item.get('date', 0))
                if year < 1400 or year > 2024:
                    continue
            except:
                continue
            
            features, palette = extract_color_features(item['image'])
            if features:
                features['year'] = year
                features['palette'] = palette
                works_data.append(features)
    
    if len(works_data) < min_works:
        print(f"Only found {len(works_data)} dated works for {artist_name} (need {min_works})")
        return None
    
    df = pd.DataFrame(works_data)
    df = df.sort_values('year').reset_index(drop=True)
    
    print(f"Collected {len(df)} dated works for {artist_name}")
    print(f"Date range: {df['year'].min()} - {df['year'].max()}")
    
    return df


# Test with a well-documented artist
print("Testing timeline extraction...")
test_df = collect_artist_timeline(dataset, 'claude-monet', min_works=10)

In [ ]:
# Collect timelines for multiple artists
ARTISTS_TO_ANALYZE = [
    'claude-monet',
    'vincent-van-gogh',
    'pablo-picasso',
    'pierre-auguste-renoir',
    'rembrandt',
    'paul-cezanne',
    'wassily-kandinsky',
    'henri-matisse',
    'edgar-degas',
    'paul-gauguin',
]

artist_timelines = {}

print("Collecting artist timelines...\n")

for artist in ARTISTS_TO_ANALYZE:
    df = collect_artist_timeline(dataset, artist, min_works=15)
    if df is not None:
        display_name = artist.replace('-', ' ').title()
        artist_timelines[display_name] = df
        print()

print(f"\nSuccessfully collected timelines for {len(artist_timelines)} artists")

## Part 2: Visualizing Individual Artist Evolution

Let's visualize how each artist's color palette evolved over their career.

In [ ]:
def plot_artist_evolution(df, artist_name, features_to_plot=None, smooth=True):
    """
    Plot how an artist's color features evolved over time.
    """
    if features_to_plot is None:
        features_to_plot = ['mean_saturation', 'mean_brightness', 'warm_ratio', 'color_diversity']
    
    n_features = len(features_to_plot)
    fig, axes = plt.subplots(n_features, 1, figsize=(14, 3*n_features), sharex=True)
    
    if n_features == 1:
        axes = [axes]
    
    years = df['year'].values
    
    for ax, feature in zip(axes, features_to_plot):
        values = df[feature].values
        
        # Scatter plot of raw data
        ax.scatter(years, values, alpha=0.4, s=50, c='steelblue', edgecolors='white', linewidth=0.5)
        
        # Smoothed trend line
        if smooth and len(values) > 5:
            # Group by year and average
            yearly_avg = df.groupby('year')[feature].mean()
            
            if len(yearly_avg) > 3:
                # Apply Gaussian smoothing
                smoothed = gaussian_filter1d(yearly_avg.values, sigma=2)
                ax.plot(yearly_avg.index, smoothed, 'r-', linewidth=2.5, label='Trend')
        
        ax.set_ylabel(feature.replace('_', ' ').title(), fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='upper right')
    
    axes[-1].set_xlabel('Year', fontsize=12)
    
    # Add career span annotation
    career_span = f"{df['year'].min()} - {df['year'].max()}"
    
    plt.suptitle(f"{artist_name}: Color Evolution ({career_span})\n{len(df)} works analyzed", 
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


# Plot evolution for each artist
if artist_timelines:
    artist_name = list(artist_timelines.keys())[0]
    df = artist_timelines[artist_name]
    plot_artist_evolution(df, artist_name)

In [ ]:
# Compare multiple artists' evolution
def plot_comparative_evolution(timelines, feature='mean_brightness'):
    """
    Compare how a specific feature evolved across multiple artists.
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    
    colors = plt.cm.Set1(np.linspace(0, 1, len(timelines)))
    
    for (artist_name, df), color in zip(timelines.items(), colors):
        # Group by year and average
        yearly_avg = df.groupby('year')[feature].mean()
        
        if len(yearly_avg) > 3:
            # Smooth
            smoothed = gaussian_filter1d(yearly_avg.values, sigma=1.5)
            ax.plot(yearly_avg.index, smoothed, '-', linewidth=2, 
                   color=color, label=artist_name, alpha=0.8)
            
            # Mark start and end
            ax.scatter([yearly_avg.index[0], yearly_avg.index[-1]], 
                      [smoothed[0], smoothed[-1]], 
                      color=color, s=100, zorder=5, edgecolors='white', linewidth=2)
    
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel(feature.replace('_', ' ').title(), fontsize=12)
    ax.set_title(f'Comparative Artist Evolution: {feature.replace("_", " ").title()}', 
                fontsize=14, fontweight='bold')
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Compare brightness evolution
if len(artist_timelines) > 1:
    plot_comparative_evolution(artist_timelines, 'mean_brightness')

In [ ]:
# Compare saturation evolution
if len(artist_timelines) > 1:
    plot_comparative_evolution(artist_timelines, 'mean_saturation')

In [ ]:
# Compare warm color usage
if len(artist_timelines) > 1:
    plot_comparative_evolution(artist_timelines, 'warm_ratio')

## Part 3: Detecting Artistic Periods

Can we automatically detect distinct periods in an artist's career (like Picasso's Blue and Rose periods)?

In [ ]:
def detect_artistic_periods(df, n_periods=3, features=None):
    """
    Use clustering to detect distinct periods in an artist's career.
    
    Returns:
        DataFrame with period labels
        Period descriptions
    """
    if features is None:
        features = ['mean_saturation', 'mean_brightness', 'warm_ratio', 
                   'color_diversity', 'mean_hue']
    
    # Prepare features
    X = df[features].values
    X = np.nan_to_num(X, nan=0.0)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Cluster
    kmeans = KMeans(n_clusters=n_periods, random_state=SEED, n_init=10)
    df = df.copy()
    df['period'] = kmeans.fit_predict(X_scaled)
    
    # Order periods chronologically (by median year)
    period_years = df.groupby('period')['year'].median().sort_values()
    period_mapping = {old: new for new, old in enumerate(period_years.index)}
    df['period'] = df['period'].map(period_mapping)
    
    # Describe each period
    period_descriptions = {}
    
    for period in range(n_periods):
        period_data = df[df['period'] == period]
        
        year_range = f"{period_data['year'].min()}-{period_data['year'].max()}"
        
        # Characterize the period
        avg_sat = period_data['mean_saturation'].mean()
        avg_bright = period_data['mean_brightness'].mean()
        avg_warm = period_data['warm_ratio'].mean()
        
        # Generate description
        sat_desc = "vibrant" if avg_sat > 40 else "muted" if avg_sat < 25 else "moderate"
        bright_desc = "bright" if avg_bright > 55 else "dark" if avg_bright < 40 else "medium-toned"
        temp_desc = "warm" if avg_warm > 0.6 else "cool" if avg_warm < 0.4 else "balanced"
        
        period_descriptions[period] = {
            'year_range': year_range,
            'n_works': len(period_data),
            'description': f"{sat_desc}, {bright_desc}, {temp_desc}",
            'saturation': avg_sat,
            'brightness': avg_bright,
            'warm_ratio': avg_warm
        }
    
    return df, period_descriptions


def visualize_periods(df, artist_name, period_descriptions):
    """
    Visualize detected artistic periods.
    """
    n_periods = len(period_descriptions)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Period colors
    period_colors = plt.cm.Set2(np.linspace(0, 1, n_periods))
    
    # 1. Timeline with periods
    ax = axes[0, 0]
    for period in range(n_periods):
        period_data = df[df['period'] == period]
        ax.scatter(period_data['year'], period_data['mean_brightness'],
                  c=[period_colors[period]], s=80, alpha=0.7,
                  label=f"Period {period+1}: {period_descriptions[period]['year_range']}",
                  edgecolors='white', linewidth=0.5)
    
    ax.set_xlabel('Year', fontsize=11)
    ax.set_ylabel('Mean Brightness', fontsize=11)
    ax.set_title('Career Timeline by Period', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # 2. Period characteristics radar
    ax = axes[0, 1]
    features_radar = ['saturation', 'brightness', 'warm_ratio']
    
    # Normalize values for radar
    for period in range(n_periods):
        values = [period_descriptions[period][f] for f in features_radar]
        # Simple normalization
        values_norm = [v/100 if f != 'warm_ratio' else v for f, v in zip(features_radar, values)]
        
        angles = np.linspace(0, 2*np.pi, len(features_radar), endpoint=False).tolist()
        values_norm += values_norm[:1]
        angles += angles[:1]
        
        ax.plot(angles, values_norm, 'o-', linewidth=2, color=period_colors[period],
               label=f"Period {period+1}")
        ax.fill(angles, values_norm, alpha=0.1, color=period_colors[period])
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([f.replace('_', ' ').title() for f in features_radar])
    ax.set_title('Period Color Profiles', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    
    # 3. Saturation vs Brightness scatter
    ax = axes[1, 0]
    for period in range(n_periods):
        period_data = df[df['period'] == period]
        ax.scatter(period_data['mean_saturation'], period_data['mean_brightness'],
                  c=[period_colors[period]], s=80, alpha=0.7,
                  label=f"Period {period+1}", edgecolors='white', linewidth=0.5)
    
    ax.set_xlabel('Mean Saturation', fontsize=11)
    ax.set_ylabel('Mean Brightness', fontsize=11)
    ax.set_title('Color Space Distribution', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # 4. Period summary table
    ax = axes[1, 1]
    ax.axis('off')
    
    table_data = []
    for period in range(n_periods):
        desc = period_descriptions[period]
        table_data.append([
            f"Period {period+1}",
            desc['year_range'],
            desc['n_works'],
            desc['description']
        ])
    
    table = ax.table(cellText=table_data,
                     colLabels=['Period', 'Years', 'Works', 'Character'],
                     loc='center',
                     cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.8)
    
    ax.set_title('Period Summary', fontsize=12, fontweight='bold', y=0.85)
    
    plt.suptitle(f'{artist_name}: Detected Artistic Periods', 
                fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


# Detect periods for artists
if artist_timelines:
    for artist_name, df in list(artist_timelines.items())[:3]:
        if len(df) >= 20:
            print(f"\nAnalyzing periods for {artist_name}...")
            df_periods, descriptions = detect_artistic_periods(df, n_periods=3)
            visualize_periods(df_periods, artist_name, descriptions)
            
            # Update timeline with periods
            artist_timelines[artist_name] = df_periods

## Part 4: Change Point Detection

Let's find the moments of dramatic stylistic shift in an artist's career.

In [ ]:
def detect_change_points(df, feature='mean_brightness', threshold=2.0):
    """
    Detect significant changes in an artist's style over time.
    
    Uses rolling statistics to find points where values deviate
    significantly from recent history.
    
    Returns:
        List of (year, change_magnitude) tuples
    """
    # Sort by year and compute yearly averages
    yearly = df.groupby('year')[feature].mean().reset_index()
    yearly = yearly.sort_values('year')
    
    if len(yearly) < 5:
        return []
    
    values = yearly[feature].values
    years = yearly['year'].values
    
    # Compute differences
    diffs = np.abs(np.diff(values))
    
    # Z-score of differences
    diff_mean = np.mean(diffs)
    diff_std = np.std(diffs)
    
    if diff_std < 1e-6:
        return []
    
    z_scores = (diffs - diff_mean) / diff_std
    
    # Find significant changes
    change_points = []
    for i, (z, year) in enumerate(zip(z_scores, years[1:])):
        if abs(z) > threshold:
            direction = "increase" if values[i+1] > values[i] else "decrease"
            change_points.append({
                'year': year,
                'z_score': z,
                'direction': direction,
                'magnitude': abs(values[i+1] - values[i])
            })
    
    return change_points


def visualize_change_points(df, artist_name, features=['mean_brightness', 'mean_saturation', 'warm_ratio']):
    """
    Visualize change points for multiple features.
    """
    fig, axes = plt.subplots(len(features), 1, figsize=(14, 3*len(features)), sharex=True)
    
    if len(features) == 1:
        axes = [axes]
    
    all_change_years = set()
    
    for ax, feature in zip(axes, features):
        # Plot data
        yearly = df.groupby('year')[feature].mean()
        ax.plot(yearly.index, yearly.values, 'b-', linewidth=2, alpha=0.7)
        ax.scatter(df['year'], df[feature], alpha=0.3, s=30, c='steelblue')
        
        # Find and mark change points
        change_points = detect_change_points(df, feature, threshold=1.5)
        
        for cp in change_points:
            color = 'red' if cp['direction'] == 'decrease' else 'green'
            ax.axvline(x=cp['year'], color=color, linestyle='--', alpha=0.7, linewidth=2)
            all_change_years.add(cp['year'])
        
        ax.set_ylabel(feature.replace('_', ' ').title(), fontsize=11)
        ax.grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Year', fontsize=12)
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', linestyle='--', label='Increase'),
        Line2D([0], [0], color='red', linestyle='--', label='Decrease')
    ]
    axes[0].legend(handles=legend_elements, loc='upper right')
    
    plt.suptitle(f'{artist_name}: Stylistic Change Points\n(Significant shifts marked)', 
                fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    return all_change_years


# Detect change points
if artist_timelines:
    for artist_name, df in list(artist_timelines.items())[:2]:
        if len(df) >= 20:
            print(f"\nAnalyzing change points for {artist_name}...")
            change_years = visualize_change_points(df, artist_name)

## Part 5: Predicting Artwork Date from Color

Can we predict when an artwork was created based solely on its color features?

In [ ]:
def build_date_predictor(df, artist_name):
    """
    Build a model to predict artwork year from color features.
    """
    feature_cols = ['mean_saturation', 'mean_brightness', 'warm_ratio',
                   'color_diversity', 'mean_hue', 'std_brightness',
                   'mean_red', 'mean_green', 'mean_blue']
    
    X = df[feature_cols].values
    y = df['year'].values
    
    # Handle NaN
    X = np.nan_to_num(X, nan=0.0)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=SEED
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=SEED, n_jobs=-1)
    model.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Feature importance
    importance = dict(zip(feature_cols, model.feature_importances_))
    
    return {
        'model': model,
        'scaler': scaler,
        'mae': mae,
        'r2': r2,
        'importance': importance,
        'y_test': y_test,
        'y_pred': y_pred,
        'feature_cols': feature_cols
    }


def visualize_date_prediction(result, artist_name):
    """
    Visualize date prediction results.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 1. Predicted vs Actual
    ax = axes[0]
    ax.scatter(result['y_test'], result['y_pred'], alpha=0.6, s=80,
              edgecolors='white', linewidth=0.5)
    
    # Perfect prediction line
    min_year = min(result['y_test'].min(), result['y_pred'].min())
    max_year = max(result['y_test'].max(), result['y_pred'].max())
    ax.plot([min_year, max_year], [min_year, max_year], 'r--', linewidth=2, label='Perfect')
    
    ax.set_xlabel('Actual Year', fontsize=12)
    ax.set_ylabel('Predicted Year', fontsize=12)
    ax.set_title(f'Date Prediction\nMAE: {result["mae"]:.1f} years, R²: {result["r2"]:.3f}', 
                fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. Feature importance
    ax = axes[1]
    importance_sorted = sorted(result['importance'].items(), key=lambda x: x[1])
    features, importances = zip(*importance_sorted)
    
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(features)))
    ax.barh([f.replace('_', ' ').title() for f in features], importances, color=colors)
    ax.set_xlabel('Importance', fontsize=12)
    ax.set_title('Feature Importance for Date Prediction', fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    plt.suptitle(f'{artist_name}: Predicting Artwork Date from Color', 
                fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


# Build predictors for each artist
if artist_timelines:
    for artist_name, df in artist_timelines.items():
        if len(df) >= 30:  # Need enough data
            print(f"\nBuilding date predictor for {artist_name}...")
            result = build_date_predictor(df, artist_name)
            print(f"  MAE: {result['mae']:.1f} years")
            print(f"  R²: {result['r2']:.3f}")
            visualize_date_prediction(result, artist_name)

## Part 6: Art Movement Timeline Analysis

Let's zoom out and look at how color trends evolved across entire art movements.

In [ ]:
def collect_movement_timeline(dataset, n_per_movement=100):
    """
    Collect artworks with dates across all movements.
    """
    movement_data = defaultdict(list)
    movement_counts = defaultdict(int)
    
    print("Collecting movement timeline data...")
    
    for item in dataset:
        style_idx = item['style']
        style_name = style_names[style_idx]
        
        if movement_counts[style_name] >= n_per_movement:
            continue
        
        try:
            year = int(item.get('date', 0))
            if year < 1400 or year > 2024:
                continue
        except:
            continue
        
        features, _ = extract_color_features(item['image'])
        if features:
            features['year'] = year
            features['style'] = style_name
            movement_data[style_name].append(features)
            movement_counts[style_name] += 1
    
    # Convert to single DataFrame
    all_data = []
    for style, data_list in movement_data.items():
        all_data.extend(data_list)
    
    df = pd.DataFrame(all_data)
    print(f"\nCollected {len(df)} dated artworks across {len(movement_data)} movements")
    
    return df


# Collect movement data
movement_df = collect_movement_timeline(dataset, n_per_movement=50)

In [ ]:
# Analyze movement timeline
if len(movement_df) > 0:
    # Average features by decade and movement
    movement_df['decade'] = (movement_df['year'] // 10) * 10
    
    # Show date distribution by movement
    fig, ax = plt.subplots(figsize=(14, 6))
    
    styles_to_plot = movement_df.groupby('style').size().sort_values(ascending=False).head(8).index
    
    for style in styles_to_plot:
        style_data = movement_df[movement_df['style'] == style]
        ax.hist(style_data['year'], bins=20, alpha=0.5, label=style)
    
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Number of Artworks', fontsize=12)
    ax.set_title('Art Movement Timeline Distribution', fontsize=14, fontweight='bold')
    ax.legend(loc='upper left', fontsize=9)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Color trends across art history
if len(movement_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    features_to_plot = ['mean_brightness', 'mean_saturation', 'warm_ratio', 'color_diversity']
    
    for ax, feature in zip(axes.flat, features_to_plot):
        # Compute decade averages
        decade_avg = movement_df.groupby('decade')[feature].agg(['mean', 'std']).reset_index()
        decade_avg = decade_avg[decade_avg['decade'] >= 1500]  # Focus on post-1500
        
        ax.fill_between(decade_avg['decade'], 
                       decade_avg['mean'] - decade_avg['std'],
                       decade_avg['mean'] + decade_avg['std'],
                       alpha=0.3, color='steelblue')
        ax.plot(decade_avg['decade'], decade_avg['mean'], 'b-', linewidth=2)
        
        ax.set_xlabel('Decade', fontsize=11)
        ax.set_ylabel(feature.replace('_', ' ').title(), fontsize=11)
        ax.set_title(f'{feature.replace("_", " ").title()} Through Art History', 
                    fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        # Add movement annotations
        movements_epochs = {
            'Renaissance': 1500,
            'Baroque': 1650,
            'Romanticism': 1820,
            'Impressionism': 1870,
            'Modern': 1920
        }
        
        for movement, year in movements_epochs.items():
            if year >= decade_avg['decade'].min() and year <= decade_avg['decade'].max():
                ax.axvline(x=year, color='gray', linestyle=':', alpha=0.5)
    
    plt.suptitle('Color Trends Across Art History (1500-2000)', 
                fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

## Part 7: Period Classification

Can we predict which period an artwork belongs to based on its color?

In [ ]:
def build_period_classifier(df, artist_name, n_periods=3):
    """
    Build a classifier to predict artistic period from color.
    """
    # First, detect periods
    df_periods, descriptions = detect_artistic_periods(df, n_periods=n_periods)
    
    feature_cols = ['mean_saturation', 'mean_brightness', 'warm_ratio',
                   'color_diversity', 'mean_hue', 'std_brightness',
                   'mean_red', 'mean_green', 'mean_blue']
    
    X = df_periods[feature_cols].values
    y = df_periods['period'].values
    
    X = np.nan_to_num(X, nan=0.0)
    
    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=SEED, stratify=y
    )
    
    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train
    model = RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)
    model.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test_scaled)
    accuracy = (y_pred == y_test).mean()
    
    return {
        'model': model,
        'scaler': scaler,
        'accuracy': accuracy,
        'y_test': y_test,
        'y_pred': y_pred,
        'descriptions': descriptions,
        'n_periods': n_periods
    }


# Build classifiers
if artist_timelines:
    for artist_name, df in artist_timelines.items():
        if len(df) >= 40:
            print(f"\nBuilding period classifier for {artist_name}...")
            result = build_period_classifier(df, artist_name, n_periods=3)
            print(f"  Accuracy: {result['accuracy']:.1%}")
            print(f"  Periods:")
            for period, desc in result['descriptions'].items():
                print(f"    Period {period+1} ({desc['year_range']}): {desc['description']}")

## Part 8: Palette Timeline Visualization

Let's create a beautiful visualization showing how an artist's actual palettes evolved.

In [ ]:
def visualize_palette_timeline(df, artist_name, n_samples=20):
    """
    Create a timeline visualization of palette evolution.
    """
    # Sample evenly across the career
    df_sorted = df.sort_values('year')
    
    if len(df_sorted) > n_samples:
        indices = np.linspace(0, len(df_sorted)-1, n_samples, dtype=int)
        df_sample = df_sorted.iloc[indices]
    else:
        df_sample = df_sorted
    
    n = len(df_sample)
    
    fig, ax = plt.subplots(figsize=(18, 4))
    
    # Plot each palette as a vertical stack
    bar_width = 0.8
    
    for i, (idx, row) in enumerate(df_sample.iterrows()):
        palette = row['palette']
        year = row['year']
        
        if palette is None:
            continue
        
        # Stack colors vertically
        for j, color in enumerate(palette[:5]):
            color_norm = tuple(c/255 for c in color)
            ax.add_patch(plt.Rectangle((i - bar_width/2, j), bar_width, 1, 
                                       facecolor=color_norm, edgecolor='white', lw=0.5))
    
    # Set up axes
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(0, 5)
    
    # Year labels
    years = df_sample['year'].values
    ax.set_xticks(range(n))
    ax.set_xticklabels(years, rotation=45, ha='right', fontsize=9)
    
    ax.set_yticks([])
    ax.set_xlabel('Year', fontsize=12)
    ax.set_title(f"{artist_name}: Palette Evolution ({years[0]} - {years[-1]})", 
                fontsize=14, fontweight='bold')
    
    # Add timeline arrow
    ax.annotate('', xy=(n-0.5, -0.5), xytext=(-0.5, -0.5),
               arrowprops=dict(arrowstyle='->', color='gray', lw=2))
    
    plt.tight_layout()
    plt.show()


# Visualize palette timelines
if artist_timelines:
    for artist_name, df in list(artist_timelines.items())[:3]:
        if 'palette' in df.columns and len(df) >= 15:
            visualize_palette_timeline(df, artist_name, n_samples=25)

## Part 9: Summary and Insights

### What We Discovered

Through temporal analysis, we've uncovered patterns in how artists and movements evolved:

1. **Individual Evolution**: Artists' palettes change systematically over their careers
2. **Distinct Periods**: Clustering reveals clear stylistic phases
3. **Change Points**: Dramatic shifts can be detected algorithmically
4. **Predictability**: Color alone can predict artwork date with surprising accuracy
5. **Historical Trends**: Art history shows long-term color trends

In [ ]:
# Summary statistics
print("\n" + "="*70)
print("TEMPORAL ANALYSIS SUMMARY")
print("="*70)

print(f"\nArtists Analyzed: {len(artist_timelines)}")

if artist_timelines:
    print("\nArtist Career Spans:")
    for artist_name, df in artist_timelines.items():
        span = df['year'].max() - df['year'].min()
        print(f"  {artist_name}: {df['year'].min()}-{df['year'].max()} ({span} years, {len(df)} works)")

if len(movement_df) > 0:
    print(f"\nMovement Timeline Data:")
    print(f"  Total artworks: {len(movement_df)}")
    print(f"  Date range: {movement_df['year'].min()}-{movement_df['year'].max()}")
    print(f"  Movements represented: {movement_df['style'].nunique()}")

## Exercises

In [ ]:
# YOUR CODE HERE

# Exercise 1: Deep dive into Picasso
# Analyze his Blue Period (1901-1904) vs Rose Period (1904-1906)
# Can you detect these automatically?

# Exercise 2: Cross-artist comparison
# Compare how Monet and Renoir evolved - they were contemporaries
# Did they influence each other?

# Exercise 3: Movement transitions
# Analyze artists who spanned multiple movements
# (e.g., Kandinsky: Impressionism → Expressionism → Abstract)

# Exercise 4: Predict movement from date + color
# Build a classifier that uses both temporal and color features

# Exercise 5: Artist aging analysis
# Normalize by career stage instead of absolute year
# Do all artists follow similar evolution patterns?


---

## Conclusion

In this final lesson, you've added the **temporal dimension** to color analysis:

1. **Tracked** individual artist palette evolution over decades
2. **Detected** distinct artistic periods automatically (Blue Period, etc.)
3. **Found** moments of dramatic stylistic shift
4. **Built** models to predict artwork date from color
5. **Analyzed** color trends across 500 years of art history

### Key Insight

Artists are not static—they **evolve**. By adding time to our analysis, we can:
- Understand artistic development and maturation
- Detect influences and pivotal moments
- Place undated works in their proper context
- See how movements influenced individual artists

This temporal perspective transforms art analysis from static snapshots to **dynamic narratives**, revealing the living, breathing evolution of artistic vision.

---

## Course Conclusion

Congratulations on completing the **Color Analysis with Renoir** course!

Over 16 lessons, you've journeyed from basic palette extraction to sophisticated machine learning applications:

| Lessons | Topics |
|---------|--------|
| 1-3 | **Fundamentals**: Palette extraction, color spaces, artist comparison |
| 4-6 | **Intermediate**: Color signatures, harmony, thematic analysis |
| 7-9 | **Advanced**: Pipelines, movement evolution, color psychology |
| 10-11 | **ML Intro**: Style classification, color naming |
| 12-13 | **Deep Learning**: Movement classification with SHAP, VAE generation |
| 14-15 | **Unsupervised**: Artist DNA embeddings, clustering, anomaly detection |
| 16 | **Temporal**: Artist evolution, period detection, historical trends |

You now have the tools to analyze art through color computationally—blending art history with data science to reveal insights invisible to the naked eye.

**Happy analyzing!**